## TensorFlow - Convolutional Neural Networks

#### Two types of deep neural networks:
<ul><li>
    Convolutional Neural Networks
</li><li>
    Recurrent Neural Networks
</li></ul>

Convolution Neural Network (CNN) are designed to process data through multiple layers of arrays.  Convolutional neural network uses three basic ideas −
<ul>
    <li>Local respective fields</li>
    <li>Convolution</li>
    <li>Pooling</li>
</ul>

## TensorFlow Implementation of CNN

### Step 1 - 
Import the necissary modules of TensorFlow and data

In [3]:
import tensorflow as tf
import numpy as np

#from tensorflow.examples.tutorials.mnist import input_data ## Old code
import tensorflow_datasets as tfds

### Step 2 - 
Declare a function called run_cnn(), which includes various parameters and optimization variables with declaration of data placeholders. These optimization variables will declare the training pattern.

In [5]:
# Construct a tf.data.Dataset
#dataset = tfds.load(name="mnist", split=tfds.Split.TRAIN)

def run_cnn():
    #mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
    mnist = tfds.load(name="mnist", split=tfds.Split.TRAIN)
    learning_rate = 0.0001
    epochs = 10
    batch_size = 50

### Step 3

Declare the training data placeholders of 28px by 28px.

In [9]:
# Use the t1 compat to make this V1 code still work
tf.compat.v1.disable_eager_execution() # Disable V2's eager execution

#x = tf.placeholder(tf.float32, [None, 784])
x = tf.compat.v1.placeholder(tf.float32, [None, 784])
x_shaped = tf.reshape(x, [-1, 28, 28, 1])
#y = tf.placeholder(tf.float32, [None, 10])
y = tf.compat.v1.placeholder(tf.float32, [None, 10])

### Step 4 − 

Now it is important to create some convolutional layers −

In [15]:
def create_new_conv_layer(
    input_data, num_input_channels, num_filters,filter_shape, pool_shape, name):
    
    conv_filt_shape = [
        filter_shape[0], filter_shape[1], num_input_channels, num_filters]
    
    #weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev = 0.03), name = name+'_W')
    weights = tf.Variable(tf.random.truncated_normal(conv_filt_shape, stddev = 0.03), name = name+'_W')
    #bias = tf.Variable(tf.truncated_normal([num_filters]), name = name+'_b')
    bias = tf.Variable(tf.random.truncated_normal([num_filters]), name = name+'_b')
    
    #Out layer defines the output
    out_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding = 'SAME')
    
    out_layer += bias
    out_layer = tf.nn.relu(out_layer)
    ksize = [1, pool_shape[0], pool_shape[1], 1]
    strides = [1, 2, 2, 1]
    out_layer = tf.nn.max_pool(
        out_layer, ksize = ksize, strides = strides, padding = 'SAME')
    
    return out_layer

layer1 = create_new_conv_layer(x_shaped, 1, 32, [5, 5], [2, 2], name = 'layer1')
layer2 = create_new_conv_layer(layer1, 32, 64, [5, 5], [2, 2], name = 'layer2')